In [19]:
import dynamicrouting_summary as dr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import oursin as urchin
import numpy as np
import pathlib
import npc_session
import npc_lims
import sqlite3
from sqlalchemy import create_engine

In [2]:
 # allow integers >8 bytes to be stored in sqlite3
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))
# ------------------------------------------------------------------------------------
DB_PATH = pathlib.Path('//allen/programs/mindscope/workgroups/dynamicrouting/dynamic_gating_insertions/dr_master.db')
# with contextlib.suppress(OSError):
#     DB_PATH.unlink()
sqlite3.connect(DB_PATH).close()
DB = f"sqlite:///{DB_PATH}"
ENGINE = create_engine(DB, echo=False)

df_ccf = pd.read_sql('channel_ccf_coords', con=ENGINE)[['session', 'MID', 'Probe', 'Implant', 'Hole']]
df_ccf

,session,MID,Probe,Implant,Hole
0,1167400342,599657,A,41,None
1,1167400342,599657,B,41,None
2,1167400342,599657,C,41,None
3,1167400342,599657,D,41,None
4,1167400342,599657,E,41,None
...,...,...,...,...,...
1207,686176_2023-12-07,686176,B,2002,F1
1208,686176_2023-12-07,686176,C,2002,F3
1209,686176_2023-12-07,686176,D,2002,D2
1210,686176_2023-12-07,686176,E,2002,E2


In [15]:
dfs = dr.get_dfs(version='v0.0.184')

In [16]:
dfs.keys()

KeysView(LazyDict(keys=['session', 'subject', 'epochs', 'trials', 'performance', 'vis_rf_mapping', 'aud_rf_mapping', 'optotagging', 'invalid_times', 'devices', 'electrode_groups', 'electrodes']))

In [17]:
electrodes_original = dfs['electrodes']
electrodes_original

FileNotFoundError: aind-scratch-data/ben.hardcastle/cache/nwb_components/v0.0.188/consolidated/session.parquet

In [22]:
import pyarrow.parquet as pq
import npc_lims

units = pq.ParquetDataset(npc_lims.get_cache_path('units'))
units.schema

num_spikes: double
firing_rate: double
presence_ratio: double
snr: double
isi_violations_ratio: double
isi_violations_count: double
rp_contamination: double
rp_violations: double
sliding_rp_violation: double
amplitude_cutoff: double
amplitude_median: double
amplitude_cv_median: double
amplitude_cv_range: double
sync_spike_2: double
sync_spike_4: double
sync_spike_8: double
firing_range: double
drift_ptp: double
drift_std: double
drift_mad: double
isolation_distance: double
l_ratio: double
d_prime: double
silhouette: double
nn_hit_rate: double
nn_miss_rate: double
peak_to_valley: double
peak_trough_ratio: double
half_width: double
repolarization_slope: double
recovery_slope: double
num_positive_peaks: int64
num_negative_peaks: int64
velocity_above: double
velocity_below: double
exp_decay: double
spread: double
electrode_group_name: string
peak_channel: int64
cluster_id: int64
default_qc: bool
amplitude: double
channels: list<element: int64>
  child 0, element: int64
unit_id: string
peak

In [32]:
electrodes_original = pq.ParquetDataset(npc_lims.get_cache_path(nwb_component='electrodes', version='v0.0.187')).read_pandas().to_pandas()
units_structures = pq.ParquetDataset(npc_lims.get_cache_path(nwb_component='units', version='v0.0.187')).read(columns=['structure', 'location', 'peak_channel', 'subject_id', 'session_idx', 'date', 
                                       'ccf_ap', 'ccf_dv', 'ccf_ml', 'default_qc']).to_pandas()

In [35]:
electrodes_original = dr.add_bool_columns(electrodes_original, version='v0.0.187')
units_structures_with_bools = dr.add_bool_columns(units_structures, version='v0.0.187')
units_structures_with_bools.dropna(inplace=True)
units_structures_with_bools.columns

Index(['structure', 'location', 'peak_channel', 'subject_id', 'session_idx',
       'date', 'ccf_ap', 'ccf_dv', 'ccf_ml', 'default_qc', 'session_id',
       'is_ephys', 'is_templeton', 'is_training', 'is_dynamic_routing',
       'is_opto'],
      dtype='object')

In [36]:
import ipywidgets

electrodes = electrodes_original.dropna()
#electrodes = electrodes[~(electrodes['session_id'].str.contains('662892'))]
electrodes = electrodes[~(electrodes['structure'].str.islower())]
electrodes = electrodes[electrodes['is_dynamic_routing'] == True]

In [37]:
units_structures_with_bools_dr = units_structures_with_bools[units_structures_with_bools['is_dynamic_routing'] == True]
units_structures_with_bools_dr.columns

Index(['structure', 'location', 'peak_channel', 'subject_id', 'session_idx',
       'date', 'ccf_ap', 'ccf_dv', 'ccf_ml', 'default_qc', 'session_id',
       'is_ephys', 'is_templeton', 'is_training', 'is_dynamic_routing',
       'is_opto'],
      dtype='object')

In [38]:
from ipywidgets import interact, Box

In [39]:
def get_subject_colors(electrodes_structures_area: pd.DataFrame) -> list[tuple[int, int, int]]: 
    subject_colors = dr.utils.generate_subject_random_colors(electrodes_structures_area)
    subject_color_list = []

    for index, row in electrodes_structures_area.iterrows():
        subject_color_list.append(subject_colors[str(row['subject_id'])])
    
    return subject_color_list

In [40]:
unique_implants = df_ccf['Implant'].unique().tolist()
unique_implants.append('All')
implants_dropdown = ipywidgets.Dropdown(options=unique_implants, value=unique_implants[-1], description='Implants', disabled=False)

unique_holes = df_ccf['Hole'].unique().tolist()
unique_holes.append('All')
holes_dropdown = ipywidgets.Dropdown(options=unique_holes, value=unique_holes[-1], description='Holes', disabled=False)

unique_probes = df_ccf['Probe'].unique().tolist()
unique_probes.append('All')
probes_dropdown = ipywidgets.Dropdown(options=unique_probes, value=unique_probes[-1], description='Probes', disabled=False)

In [69]:
def update_3d_viewer_based_on_trajectory():
    urchin.ccf25.clear()
    urchin.clear_meshes()

    if implants_dropdown.value == 'All':
        pmeshes = urchin.meshes.create(len(electrodes)) #creates 2 primitives, stored in list pmeshes
        electrodes_sessions = electrodes
        coords = electrodes[['x', 'z', 'y']].to_numpy().tolist()
    else:
        probe_target_sessions = df_ccf[(df_ccf['Implant'] == implants_dropdown.value) & (df_ccf['Hole'] == holes_dropdown.value)
                                    & (df_ccf['Probe'] == probes_dropdown.value)]['session'].tolist()
        probe_target_sessions_filtered = []
        for session in probe_target_sessions:
            try:
                session_record = npc_session.SessionRecord(session)
                probe_target_sessions_filtered.append(f"{session_record.subject}_{session_record.date}_{session_record.idx}")
            except ValueError:
                pass
        
        electrodes_sessions = electrodes[electrodes['session_id'].isin(probe_target_sessions_filtered)]
        electrodes_sessions = electrodes_sessions[electrodes_sessions['group_name'] == f"probe{probes_dropdown.value}"]
        pmeshes = urchin.meshes.create(len(electrodes_sessions)) #creates 2 primitives, stored in list pmeshes
        coords = electrodes_sessions[['x', 'z', 'y']].to_numpy().tolist()
   
    
    print(f"Implant {implants_dropdown.value} Hole {holes_dropdown.value} Probe {probes_dropdown.value}")
    print(f"Sessions: {electrodes_sessions['session_id'].unique().tolist()}")
    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    coords_list = [[x[0], x[1], x[2]] for x in coords]
    sizes_list = [[0.07,0.07,0.07]]*len(coords)

    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, get_subject_colors(electrodes_sessions))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas(electrodes_sessions['structure'].unique().tolist())
    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")

(URN) disconnected from server
(URN) connected to server
Login sent with ID: 7f437664, copy this ID into the renderer to connect.


In [53]:
def filter_by_implant(implant: str):
    if implant == 'All':
        holes_dropdown.options = unique_holes
        holes_dropdown.value = 'All'
    else:
        df_implant = df_ccf[df_ccf['Implant'] == implant]
        holes_dropdown.options = df_implant['Hole'].unique().tolist()
        holes_dropdown.value = holes_dropdown.options[0]

In [54]:
def filter_by_implant_hole(implant: str, hole: str):
    if implant == 'All' and hole == 'All':
        probes_dropdown.options = unique_probes
        probes_dropdown.value = 'All'
    else:
        df_implant_hole = df_ccf[(df_ccf['Implant'] == implant) & (df_ccf['Hole'] == hole)]
        probes_dropdown.options = df_implant_hole['Probe'].unique().tolist()
        probes_dropdown.value = probes_dropdown.options[0]

In [44]:
urchin.setup()

(URN) connected to server
Login sent with ID: 7f437664, copy this ID into the renderer to connect.


(URN) disconnected from server
(URN) connected to server
Login sent with ID: 7f437664, copy this ID into the renderer to connect.


In [45]:
urchin.ccf25.load()

In [46]:
@interact(x=['coronal', 'axial', 'sagittal'])
def camera_rotation(x):
    urchin.camera.main.set_rotation(x)
    urchin.camera.main.set_mode('perspective')
    urchin.camera.main.set_background_color('#ffffff')

interactive(children=(Dropdown(description='x', options=('coronal', 'axial', 'sagittal'), value='coronal'), Ou…

In [47]:
@interact(x=(5, 60))
def zoom(x=45):
    urchin.camera.main.set_zoom(60-x)

interactive(children=(IntSlider(value=45, description='x', max=60, min=5), Output()), _dom_classes=('widget-in…

In [48]:
items = [implants_dropdown, holes_dropdown, probes_dropdown]
box = Box(children=items)
box

Box(children=(Dropdown(description='Implants', index=8, options=('41', 'football', 'TS1', 'TS5', '2002', 'temp…

In [62]:
filter_by_implant(implants_dropdown.value)

In [63]:
filter_by_implant_hole(implants_dropdown.value, holes_dropdown.value)

In [70]:
update_3d_viewer_based_on_trajectory()

Implant 2002 Hole C1 Probe C
Sessions: ['644867_2023-02-20_0', '649943_2023-02-13_0', '660023_2023-08-09_0', '666986_2023-08-16_0', '666986_2023-08-17_0', '667252_2023-09-25_0', '686740_2023-10-25_0', '686740_2023-10-26_0']


In [ ]:
unique_areas = electrodes['structure'].unique()
unique_areas.sort()
area_dropdown = ipywidgets.Dropdown(options=unique_areas, value=unique_areas[0], description='CCF Structure Area', disabled=False)
area_dropdown

@interact(x=unique_areas)
def update_3d_viewer(x):
    urchin.ccf25.clear()
    urchin.clear_meshes()
    units_structures_with_bools_dr_area = units_structures_with_bools_dr[units_structures_with_bools_dr['structure'] == x]
    electrodes_structures_area = electrodes[electrodes['structure'] == x]

    print(f"{x}: {len(electrodes_structures_area['subject_id'].unique())} mice")
    print(f"{x}: {len(electrodes_structures_area['session_id'].unique())} sessions")
    print(f"{x}: {len(units_structures_with_bools_dr_area)} units")

    pmeshes = urchin.meshes.create(len(electrodes_structures_area)) #creates 2 primitives, stored in list pmeshes
    coords = electrodes_structures_area[['x', 'z', 'y']].to_numpy().tolist()


    urchin.ccf25.root.set_visibility(True)
    urchin.ccf25.root.set_material('transparent-lit')
    urchin.ccf25.root.set_alpha(0.15)
    urchin.ccf25.root.set_color("#000000")

    # reorder to AP/ML/DV for Urchin and make a list of lists
    coords_list = [[x[0], x[1], x[2]] for x in coords]
    sizes_list = [[0.07,0.07,0.07]]*len(coords)

    urchin.meshes.set_positions(pmeshes,coords_list) #sets the positions of the primitives
    urchin.meshes.set_colors(pmeshes, get_subject_colors(electrodes_structures_area))
    urchin.meshes.set_scales(pmeshes, sizes_list)

    area_list = urchin.ccf25.get_areas([x])

    urchin.ccf25.set_visibilities(area_list, True, urchin.utils.Side.LEFT)
    urchin.ccf25.set_materials(area_list, 'transparent-unlit', "left")
    urchin.ccf25.set_alphas(area_list, 0.2, "left")


In [ ]:
counts = {'area': [], 'num_units': [], 'num_sessions': [], 'num_subjects': []}
units_structure_areas_no_white_matter = units_structures_with_bools_dr[~(units_structures_with_bools_dr['structure'].str.islower())]
for area in units_structure_areas_no_white_matter['structure'].unique():
    units_structure_areas = units_structure_areas_no_white_matter[units_structure_areas_no_white_matter['structure'] == area]
    counts['area'].append(area)
    counts['num_sessions'].append(len(units_structure_areas['session_id'].unique()))
    counts['num_subjects'].append(len(units_structure_areas['subject_id'].unique()))
    counts['num_units'].append(len(units_structure_areas))

df_counts = pd.DataFrame(counts)

In [ ]:
df_counts.style.applymap(lambda x: 'background-color : red' if x>=3 else '', subset=['num_subjects']).applymap(lambda x: 'background-color : red' if x>150 else '', subset=['num_units'])